## 00 Import Modules

In [1]:
#!pip install --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

In [2]:
import os
import pathlib
import torch
import numpy as np
import textwrap

from random import randint
from itertools import zip_longest
from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [4]:
#url = 'https://huggingface.co/Qwen/Qwen2.5-0.5B'
#model_name = url.split('.co/')[-1]

model_name = 'Qwen/Qwen2.5-0.5B-Instruct'

In [5]:
def load_model(model_name, base = True):
  if base == True:
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype = torch.float16,
      trust_remote_code = True
    ).to(device)

    return model
    
  else:
    bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_quant_type = 'nf4',
      bnb_4bit_compute_dtype = torch.float16,
      bnb_4bit_use_double_quant = True,
    )
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config = bnb_config,
      trust_remote_code = True
    ).to(device)

    return model

In [6]:
model = load_model(model_name, base = False)
model

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [7]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 315119488
Trainable parameters : 136178560
Trainable percentage: 43.21%


## 02 Import Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## 03 Import Dataset

In [9]:
#url = 'https://huggingface.co/datasets/KingNish/reasoning-base-20k'
#dataset_name = url.split('datasets/')[-1]

dataset_name = 'mlabonne/FineTome-100k'

In [10]:
max_length = 1024

In [11]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [12]:
dataset = dataset.select(range(10000))

In [13]:
dataset.select(range(5)).to_pandas().head()

,conversations,source,score
0,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.212621
1,"[{'from': 'human', 'value': 'Explain how recur...",infini-instruct-top-500k,5.157649
2,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.147540
3,"[{'from': 'human', 'value': 'Explain the conce...",infini-instruct-top-500k,5.053656
4,"[{'from': 'human', 'value': 'Print the reverse...",infini-instruct-top-500k,5.045648


In [14]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [15]:
features = list(dataset.features.keys())
print(features)

['conversations', 'source', 'score']


## 04 Text Formatting

In [16]:
def transform_conversations(example):
  role_map = {
    'human' : 'user',
    'gpt' : 'assistant'
  }

  transformed_conversations = [
    {
      'role' : role_map.get(turn['from'], turn['from']),
      'content' : turn['value']
    }
    for turn in example['conversations']
  ]
  return {'conversations': transformed_conversations}

In [17]:
formatted_dataset = dataset.map(transform_conversations, remove_columns = features)
formatted_dataset

Dataset({
    features: ['conversations'],
    num_rows: 10000
})

In [18]:
print(formatted_dataset[0]['conversations'])

[{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.', 'role': 'user'}, {'content': 'Boolean operators are 

In [19]:
def format_conversation(example):
  for entry in example['conversations']:
    role = entry['role']
    content = entry['content']

    if role == 'user':
      formatted_text = f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"
    elif role == 'assistant':
      formatted_text += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"

  return {'prompt': formatted_text}

In [20]:
formatted_dataset = formatted_dataset.map(
  format_conversation,
  remove_columns = list(formatted_dataset.features.keys())
)
formatted_dataset

Dataset({
    features: ['prompt'],
    num_rows: 10000
})

In [21]:
print(formatted_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. 

Furthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.

Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.
<|eot_id|><|start_header_id|>a

## 05 Tokenization

In [22]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [23]:
tokenized_dataset = formatted_dataset.map(tokenize_data, batched = True)#, remove_columns = 'text')
tokenized_dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [24]:
print(tokenized_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. 

Furthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.

Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.
<|eot_id|><|start_header_id|>a

In [25]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [26]:
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [27]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,<|start_header_id|>user<|end_header_id|>\n\nWr...,"[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,<|start_header_id|>user<|end_header_id|>\n\nWh...,"[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [28]:
print(train_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

How do you graph the inequality #x > -4#?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

To graph the inequality #x > -4#, follow these steps:

1. Start by plotting the boundary line: Draw a vertical line at #x = -4# on the X-axis. This line divides the X-axis into two halves.

2. Identify the direction of the inequality: Since the inequality is #x > -4#, it indicates that we are looking for all values of #x# that are greater than -4.

3. Shade the appropriate region: Shade the entire region to the right of the line at #x = -4#. This shaded area represents all the points whose X-coordinate is greater than -4.

4. Include a dashed line: Since the inequality is "greater than" (#>,# not "#≥#"), draw the line at #x = -4# as a dashed line, indicating that the points on the line itself are not included in the solution.

The graph consists of the entire half-plane to the right of the dashed vertical line at #x = -4#, excluding the line itsel

In [29]:
print(train_dataset[0]['input_ids'])

[27, 91, 2468, 8757, 842, 91, 29, 872, 27, 91, 408, 8757, 842, 91, 1339, 4340, 653, 498, 4771, 279, 31205, 671, 87, 861, 481, 19, 2, 5267, 27, 91, 68, 354, 842, 91, 1784, 91, 2468, 8757, 842, 91, 29, 77091, 27, 91, 408, 8757, 842, 91, 1339, 1249, 4771, 279, 31205, 671, 87, 861, 481, 19, 60778, 1795, 1493, 7354, 1447, 16, 13, 5145, 553, 43902, 279, 18732, 1555, 25, 11992, 264, 12140, 1555, 518, 671, 87, 284, 481, 19, 2, 389, 279, 1599, 35321, 13, 1096, 1555, 64828, 279, 1599, 35321, 1119, 1378, 74112, 382, 17, 13, 64547, 279, 5106, 315, 279, 31205, 25, 8704, 279, 31205, 374, 671, 87, 861, 481, 19, 60778, 432, 14807, 429, 582, 525, 3330, 369, 678, 2750, 315, 671, 87, 2, 429, 525, 7046, 1091, 481, 19, 382, 18, 13, 79818, 279, 8311, 5537, 25, 79818, 279, 4453, 5537, 311, 279, 1290, 315, 279, 1555, 518, 671, 87, 284, 481, 19, 87846, 1096, 91766, 3082, 10868, 678, 279, 3501, 6693, 1599, 80697, 374, 7046, 1091, 481, 19, 382, 19, 13, 29734, 264, 66722, 1555, 25, 8704, 279, 31205, 374, 330, 652

In [30]:
print(train_dataset[0]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## 06 Data Collator Set Up

In [31]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 07 Evaluation Metrics Set Up

In [32]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [33]:
torch.cuda.empty_cache()

## 08 Set Up PEFT / LoRA / QLoRA

In [34]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [35]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'LoRA')
peft_model.print_trainable_parameters()

trainable params: 35,192,832 || all params: 529,225,600 || trainable%: 6.6499


## 09 Training Model

In [36]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
            (lora_dropout): ModuleDict(
              (LoRA): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (LoRA): Linear(in_features=896, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (LoRA): Linear(in_features=64, out_features=896, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=896, out_features=128, bias=True)
            (lora_dropout): ModuleDict(
              (LoRA):

In [37]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 350312320
Trainable parameters : 35192832
Trainable percentage: 10.05%


In [38]:
torch.cuda.empty_cache()

In [39]:
save_path = './model'

batch_size = 2
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=20,
eval_strategy=steps,
eval_use_gather_object=F

In [40]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

In [41]:
trainer.train()

Currently training with a batch size of: 2
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 200
  Number of trainable parameters = 35,192,832
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more informati

Step,Training Loss,Validation Loss
20,1.324000,1.312997
40,1.295800,1.207026
60,1.166500,1.095321
80,1.076700,1.045753
100,1.054400,1.020004
120,0.974900,1.009685
140,1.072500,1.005408
160,1.005700,1.003242
180,0.997800,1.002476
200,0.999300,1.002378


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a c

TrainOutput(global_step=200, training_loss=1.0967500305175781, metrics={'train_runtime': 1466.0415, 'train_samples_per_second': 1.091, 'train_steps_per_second': 0.136, 'total_flos': 4210200831590400.0, 'train_loss': 1.0967500305175781, 'epoch': 0.17777777777777778})

## 10 Model Evaluation

In [42]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


Evaluation Results: {'eval_loss': 1.0023784637451172, 'eval_runtime': 44.9971, 'eval_samples_per_second': 4.445, 'eval_steps_per_second': 1.111, 'epoch': 0.17777777777777778}


## 11 Save Model

In [43]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-In

## 12 Load PEFT Model

In [44]:
torch.cuda.empty_cache()

In [45]:
peft_path = save_path + '/LoRA'
peft_path

'./model/LoRA'

In [46]:
peft_model = PeftModel.from_pretrained(model, peft_path)

## 13 Reload & Recheck Base Model

In [47]:
model = load_model(model_name, base = False)
model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775/config.json
Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

CUDA backend validation successful.
Overriding torch_dtype=Non

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [48]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 315119488
Trainable parameters : 136178560
Trainable percentage: 43.21%


In [49]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (LoRA): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (LoRA): Linear(in_features=896, out_features=64, bias=False)
                  (default): Linear(in_features=896, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (LoRA): Linear(in_features=64, out_features=896, bias=False)
                  (default): Linear(in_features=64, out_features=896, bias=False)
        

In [50]:
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 385505152
Trainable parameters : 0
Trainable percentage: 0.00%


## 13 Pre Test & Post Test

In [59]:
def pre_assistant(prompt):
  messages = [
    {'role' : 'system', 'content' : 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'},
    {'role' : 'user', 'content': prompt}
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
    truncation = True,
    padding = True,
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0])#, skip_special_tokens = True)

In [60]:
def post_assistant(prompt):
  messages = [
    {'role' : 'system', 'content' : 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'},
    {'role' : 'user', 'content': prompt}
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
    truncation = True,
    padding = True,
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0])#, skip_special_tokens = True)

In [61]:
def print_side_by_side(pre_text, post_text, width = 50):
  pre_wrapped = textwrap.wrap(pre_text, width)
  post_wrapped = textwrap.wrap(post_text, width)
    
  print('PRE-TEST'.center(width), ' | ', 'POST-TEST'.center(width))
  print(
    str(sum(p.numel() for p in model.parameters())).center(width),
    '|',
    str(sum(p.numel() for p in peft_model.parameters())).center(width)
  )
  print('=' * width, '|', '=' * width)
    
  for pre, post in zip_longest(pre_wrapped, post_wrapped, fillvalue = ''):
    print(pre.ljust(width), ' | ', post.ljust(width))

In [62]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
<|im_start|>system You are Qwen, created by         |  <|im_start|>system You are Qwen, created by       
Alibaba Cloud. You are a helpful                    |  Alibaba Cloud. You are a helpful                  
assistant.<|im_end|> <|im_start|>user Describe how  |  assistant.<|im_end|> <|im_start|>user Describe how
airplanes maintain altitude.<|im_end|>              |  airplanes maintain altitude.<|im_end|>            
<|im_start|>assistant Airplanes maintain altitude   |  <|im_start|>assistant Airplanes maintain altitude 
through a combination of aerodynamic principles     |  through a combination of aerodynamic principles   
and mechanical systems. Here's a detailed         

In [66]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
<|im_start|>system You are Qwen, created by         |  <|im_start|>system You are Qwen, created by       
Alibaba Cloud. You are a helpful                    |  Alibaba Cloud. You are a helpful                  
assistant.<|im_end|> <|im_start|>user Provide a     |  assistant.<|im_end|> <|im_start|>user Provide a   
comprehensive list of benefits of integrating a     |  comprehensive list of benefits of integrating a   
focus on student questioning into the school        |  focus on student questioning into the school      
curriculum with a clear emphasis on the             |  curriculum with a clear emphasis on the           
correlation between student question formulation  

In [67]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
<|im_start|>system You are Qwen, created by         |  <|im_start|>system You are Qwen, created by       
Alibaba Cloud. You are a helpful                    |  Alibaba Cloud. You are a helpful                  
assistant.<|im_end|> <|im_start|>user What is the   |  assistant.<|im_end|> <|im_start|>user What is the 
observable universe, and why does its size not      |  observable universe, and why does its size not    
correspond to the age of the universe multiplied    |  correspond to the age of the universe multiplied  
by the speed of light? How does the expansion of    |  by the speed of light? How does the expansion of  
the universe affect our perception of its         

In [71]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
<|im_start|>system You are Qwen, created by         |  <|im_start|>system You are Qwen, created by       
Alibaba Cloud. You are a helpful                    |  Alibaba Cloud. You are a helpful                  
assistant.<|im_end|> <|im_start|>user How can       |  assistant.<|im_end|> <|im_start|>user How can     
chemistry students effectively learn and apply      |  chemistry students effectively learn and apply    
experimental techniques when faced with limited     |  experimental techniques when faced with limited   
opportunities for hands-on experimentation in the   |  opportunities for hands-on experimentation in the 
classroom setting?<|im_end|> <|im_start|>assistant

In [76]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
<|im_start|>system You are Qwen, created by         |  <|im_start|>system You are Qwen, created by       
Alibaba Cloud. You are a helpful                    |  Alibaba Cloud. You are a helpful                  
assistant.<|im_end|> <|im_start|>user What are the  |  assistant.<|im_end|> <|im_start|>user What are the
key factors that contribute to the formation and    |  key factors that contribute to the formation and  
maintenance of chaparral ecosystems, and how do     |  maintenance of chaparral ecosystems, and how do   
these ecosystems support their unique species       |  these ecosystems support their unique species     
assemblages?<|im_end|> <|im_start|>assistant The  